In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from bs4 import BeautifulSoup
import csv

In [2]:
##Step 1: Login to Linkedin 
# Open Chrome and login linkedin login site
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()
url = "https://www.linkedin.com/login"
driver.get(url)
sleep(2)

# Import username and password
credential = open('login_credential.txt')
line = credential.readlines()
username = line[0]
password = line[1]

# key in username/password --> login
driver.find_element_by_xpath("//input[@id = 'username']").send_keys(username)
sleep(2)
driver.find_element_by_xpath("//input[@id = 'password' ]").send_keys(password)
sleep(2)
driver.find_element_by_xpath("//button[@aria-label= 'Sign in' ]").click()
sleep(2)

## Step 2: Search for the profile we want to crawl
search_field =  driver.find_element_by_xpath("//input[@aria-label = 'Search']")
search_field.send_keys("Automation test")
search_field.send_keys(Keys.RETURN)
sleep(10)
driver.find_element_by_xpath("//nav[@aria-label = 'Search filters']//button[text()= 'People']").click()
sleep(5)

## Step 3: Scrape the URLs of the profiles

def  GetURL():
    page_source = BeautifulSoup(driver.page_source)
    profiles =  page_source.find_all('a', class_= 'app-aware-link scale-down')
    all_profile_URL = []
    for profile in profiles:
        profile_ID = profile.get('href')
        all_profile_URL.append(profile_ID)
    return all_profile_URL


input_page = int(input("How many pages you want to scrape: "))
URLs_all_page = []
for page in range(input_page):
    URLs_one_page = GetURL()
    sleep(5)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    sleep(2)
    next_button =  driver.find_element_by_xpath("//button[@aria-label = 'Next']")
    sleep(2)
    driver.execute_script("arguments[0].click();", next_button)
    URLs_all_page = URLs_all_page + URLs_one_page
    sleep(2)


print(URLs_all_page)

  

[WDM] - Downloading: 100%|██████████| 6.80M/6.80M [01:27<00:00, 81.5kB/s]


['https://www.linkedin.com/in/ni-ho-thi-mi-51b0b7267?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAEFVHScB0GF2Dz3nRAOEW06sKF-pHwJ2pvw', 'https://www.linkedin.com/in/phuongnd1306?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADUbstsB_ijKytRhj7oFTm0oqxoIyl6WnV4', 'https://www.linkedin.com/in/hoa-nguyen-01121516b?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAChuus0BJ0dScF3Pq7wMIuv-XYEZwHILcAQ', 'https://www.linkedin.com/in/pham-thi-be-hien?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADEq110BFvSJVi65HsXgNKjctlmHWZYGF_s', 'https://www.linkedin.com/in/tuananhle-927829237?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADsIc-8Bl5TomOojkIZPoePgLPhI7EmCtKg', 'https://www.linkedin.com/in/phuc-dinh-027341212?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADXMjt8BtLtQcs5vdCbMc9J8rHdrLXUo9fs', 'https://www.linkedin.com/in/quyenanhtp1609?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADQCxWgB_JojsZKR_wRAyNrwyu5nTEQF5CI', 'https://www.linkedin.com/in/thanh-phan-19843239?miniProfileUrn=urn%3Ali

In [3]:
## Step 4: Scrape the data of 1 linkedin profile, and write the data to a .CSV file
with open('output.csv', 'w', newline= '')as file_output: 
    headers = ['Name', 'Job Title', 'Location', 'URL']
    writer = csv.DictWriter(file_output, delimiter = ',', lineterminator = '\n', fieldnames = headers)
    writer.writeheader() 

    for linkedin_URL in URLs_all_page:
        driver.get(linkedin_URL)
        sleep(5)
        page_source = BeautifulSoup(driver.page_source, "html.parser")
        infor_div = page_source.find('div', class_="mt2 relative")
        infor_loc = infor_div.find_all('div', class_ = 'pv-text-details__left-panel')
        name = infor_loc[0].find('h1').get_text()
        print('Profile name is: ',name)
        title = infor_loc[0].find('div', class_ = 'text-body-medium break-words').get_text().strip()
        print('Profile title is: ',title)
        location = infor_loc[1].find('span').get_text().strip()     
        print('Profile location is: ',location)
        writer.writerow({headers[0]:name,headers[1]:title, headers[2]: location, headers[3]: linkedin_URL}) 
        print('\n')

    

Profile name is:  Ni Ho Thi Mi
Profile title is:  Test Automation Engineer at NashTech
Profile location is:  Ho Chi Minh City, Vietnam


Profile name is:  Duc Phuong Nguyen
Profile title is:  Automation Test Engineer
Profile location is:  Hanoi, Hanoi, Vietnam


Profile name is:  Hoa Nguyen
Profile title is:  Automation Test Engineer
Profile location is:  Hanoi, Hanoi, Vietnam


Profile name is:  Hien Pham
Profile title is:  Automation Test Engineer
Profile location is:  Ho Chi Minh City, Vietnam


Profile name is:  Tuan Anh Le
Profile title is:  Automation Test Engineer | Quality Assurance Automation Engineer
Profile location is:  Ba Đình District, Hanoi, Vietnam


Profile name is:  Phuc Dinh
Profile title is:  Automation Test Engineer at Hansen Technologies
Profile location is:  Ho Chi Minh City, Vietnam


Profile name is:  Anh Trần Phúc Quyền
Profile title is:  Senior Automation Test Engineer & Automation Test Lead
Profile location is:  Ho Chi Minh City, Vietnam


Profile name is:  